In [ ]:
import os
import logging
from typing import Optional
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI, HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration (accessing API keys from environment variables)#free and paid are both 
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
SAMPLE_WRITINGS_PATH = "sample_writings.txt"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

def load_sample_writings(file_path: str) -> str:
    """Load the author's sample writings from a text file."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            sample_writings = f.read()
        logger.info("Sample writings loaded successfully.")
        return sample_writings
    except FileNotFoundError:
        logger.error(f"File not found: {file_path}")
        raise
    except Exception as e:
        logger.error(f"Error loading sample writings: {e}")
        raise

def split_text(text: str, chunk_size: int, chunk_overlap: int) -> list:
    """Split text into chunks for embedding."""
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text)
    logger.info(f"Text split into {len(chunks)} chunks.")
    return chunks

def create_vector_store(texts: list, embedding_model: Optional[str] = "openai") -> Chroma:
    """Create a vector store from text chunks using specified embedding model."""
    if embedding_model == "openai":
        embeddings = OpenAIEmbeddings()
    elif embedding_model == "huggingface":
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    else:
        logger.error(f"Unsupported embedding model: {embedding_model}")
        raise ValueError(f"Unsupported embedding model: {embedding_model}")
    
    vector_store = Chroma.from_texts(texts, embeddings)
    logger.info("Vector store created successfully.")
    return vector_store

def initialize_llm(model: str = "openai") -> Optional:
    """Initialize the Language Model based on the specified type."""
    if model == "openai":
        return OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
    elif model == "huggingface":
        # Use a specific model from Hugging Face Hub
        return HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0.1, "max_length":512}, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN) 
    else:
        logger.error(f"Unsupported LLM model: {model}")
        raise ValueError(f"Unsupported LLM model: {model}")

def create_retrieval_qa(llm, retriever, memory) -> RetrievalQA:
    """Create a RetrievalQA chain with memory."""

    # Use a prompt template for better control over the LLM's output
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    Helpful Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,  # Optionally return source docs for context
        memory = memory, # Add memory to the chain
        prompt=PROMPT # Use the prompt template
    )
    logger.info("RetrievalQA chain created successfully.")
    return qa_chain

def main():
    # Load sample writings
    sample_writings = load_sample_writings(SAMPLE_WRITINGS_PATH)
    
    # Split text into chunks
    texts = split_text(sample_writings, CHUNK_SIZE, CHUNK_OVERLAP)
    
    # Create vector store with desired embedding model
    embedding_model = "openai"  # Change to "huggingface" for free embeddings
    docsearch = create_vector_store(texts, embedding_model=embedding_model)
    
    # Initialize Language Model 
    llm_model = "huggingface"  # Or "openai"
    llm = initialize_llm(model=llm_model)

    # Initialize memory
    memory = ConversationBufferMemory(memory_key="chat_history", input_key='question') 

    # Create RetrievalQA chain with memory
    qa = create_retrieval_qa(llm, docsearch.as_retriever(), memory)

    # Start interactive session
    while True:
        query = input("Enter your question (or 'exit' to quit): ")
        if query.lower() == "exit":
            break

        try:
            response = qa.run(query)
            print("AI Author Bot Response:")
            print(response)
        except Exception as e:
            logger.error(f"Error during QA run: {e}")

if __name__ == "__main__":
    main()